# GenAI with Python: Coding Agents

###### [Article: TowardsDataScience](https://towardsdatascience.com/genai-with-python-coding-agents-765406d26584)

### 0 - Setup

##### Data

In [ ]:
import pandas as pd
from jupyterlab.semver import regexp
from litellm import api_base
from litellm.utils import executor

dtf = pd.read_csv("data_titanic.csv")
dtf.head(3)

In [ ]:
print("How many people in dataset? -->", len(dtf))
print("How many people survived? -->", len(dtf[dtf["Survived"]==1]))
print("How many people died? -->", len(dtf[dtf["Survived"]==0]))

##### LLM

In [ ]:
from langchain_ollama import OllamaLLM #0.2.12
import regex as re
from crewai import LLM


# LLM
llm = OllamaLLM(model="codestral", verbose=True)
# llm = OllamaLLM(model="qwen2.5", verbose=True)
crewai_llm = LLM(model="ollama/qwen2.5")

res = llm.invoke(input=['''Answer short with single number: Do you know 'Titanic' dataset from Kaggle? If yes, tell me how many people survived - column ["Survived"]==1?''']);
print("\n",res)
p = re.compile(r'^.*(?:.*surviv.*)?\s(\d+)[*\s]*(?:\w*\s*)*\"?surviv.*$', re.IGNORECASE & re.MULTILINE)
survived = p.findall(res)
print(survived)

##### DB

In [ ]:
import sqlite3

dtf.to_sql(index=False, name="titanic", con=sqlite3.connect("database.db"), if_exists="replace")

In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///database.db")

### 1 - SQL Agent

##### Tools

In [ ]:
## DB Connection
from crewai_tools import tool #0.8.3
from langchain_community.tools import ListSQLDatabaseTool, InfoSQLDatabaseTool, QuerySQLDatabaseTool

@tool("tool_tables")
def tool_tables() -> str:
    """Get all the tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

@tool("tool_schema")
def tool_schema(tables: str) -> str:
    """Get table schema. Example Input: table1, table2, table3"""
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

@tool("tool_query")
def tool_query(sql: str) -> str:
    """Execute a SQL query"""
    return QuerySQLDatabaseTool(db=db).invoke(sql)

print("--- Get Tables ---")
print( tool_tables.run() )

print("\n--- Get Schema ---")
print( tool_schema.run( tables=tool_tables.run() ) )

print("\n--- SQL Query ---")
print( tool_query.run(f"SELECT * FROM {tool_tables.run()} LIMIT 3") )

In [ ]:
## LLM Checking SQL
from langchain_community.tools.sql_database.tool import QuerySQLCheckerTool

@tool("tool_check")
def tool_check(sql: str) -> str:
    """
    Before executing a query, always use this tool to review the SQL query 
    and correct the code if necessary.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query":sql})

tool_check.run(f"SELECT * FROM {tool_tables.run()} LIMIT 3 WHRE id=5").split('\n')[0]

#### Agent

In [ ]:
import crewai #0.86.0

prompt = '''Extract data with SQL query to answer {user_input}'''

## Agent
agent_sql = crewai.Agent(
    role="Database Engineer",
    goal=prompt,
    backstory='''
        You are an experienced database engineer that creates and optimize efficient SQL queries.
        Use the `tool_tables` to find tables.
        Use the `tool_schema` to get the metadata for the tables.
        Use the `tool_check` to review your queries before executing.
        Use the `tool_query` to execute SQL queries.
     ''',
    tools=[tool_tables, tool_schema, tool_query, tool_check], 
    max_iter=10,
    llm=crewai_llm,
    allow_delegation=False, 
    verbose=True)

## Task
task_sql = crewai.Task(
    description=prompt,
    agent=agent_sql,
    expected_output='''Output of the query'''
)

#### Test single Agent

In [ ]:
crew = crewai.Crew(agents=[agent_sql], tasks=[task_sql], verbose=False)
res = crew.kickoff(inputs={"user_input":"how many people died?"})
print("\nResponse:\n", res)

### 2 - Python Agent

##### Tools

In [ ]:
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import Tool

tool_pycode = Tool(name="tool_pycode",
    description='''
    A Python shell. Use this to execute python commands. Input should be a valid python command. 
    If you want to see the output of a value, you should print it out with `print(...)`.
    ''',
    func=PythonREPL().run)

result = tool_pycode.run("import numpy as np; print(np.sum([1,2]))")
print(f'\n{result}')

In [ ]:
## LLM Checking Python

@tool("tool_eval")
def tool_eval(code: str) -> str:
    """
    Before executing Python code, always use this tool to evaluate code and correct the code if necessary.
    Example: `import numpy as np print(np.sum([1,2]))` would give an error, 
    so you must change it to `import numpy as np; print(np.sum([1,2]))`
    """
    res = llm.invoke(input=['''review the following python code and correct it if you find errors.
    You must return very short answer in json format {"answer":<answer>, "code":<corrected code>}:\n''' + code]).split("\n")
    return res

print( tool_eval.run("print(Res:')") )

##### Agent

In [ ]:
prompt = '''You analyze the data received from the database engineer to answer {user_input}'''
# crewai_llm = LLM(
#     model="ollama/deepseek-coder-v2"
# )
## Agent
agent_py = crewai.Agent(
    role="Data Analyst",
    goal=prompt,
    backstory='''
        You are an experienced data anlyst that analyzes datasets using Python.
        You have attention to detail and always produce very clear and detailed results.
        First generate Python code you need to analyze the data.
        Then use the `tool_eval` to check your code.
        Finally use `tool_pycode` to execute the code and return the output.
    ''',
    tools=[tool_eval, tool_pycode],
    max_iter=10,
    llm=crewai_llm,
    allow_delegation=False, verbose=True)

## Task
task_py = crewai.Task(
    description=prompt,
    agent=agent_py,
    context=[task_sql],
    expected_output='''Output of Python code''')

##### Test single Agent

In [ ]:
import os
os.environ['LITELLM_LOG'] = 'DEBUG' # litellm.set_verbose=True
crew = crewai.Crew(agents=[agent_py], tasks=[task_py], verbose=True)
res = crew.kickoff(inputs={"user_input": f"how many people died in this dataset? {dtf.to_string()}"})

### 3 - HTML Agent

##### Tools

In [ ]:
#no need

##### Agent

In [ ]:
prompt = '''You write executive summary reports based on the work of the data analyst to answer {user_input}'''

## Agent
agent_html = crewai.Agent(
    role="Web Developer",
    goal=prompt,
    backstory='''
        You are an experienced web developer that writes beautiful reports using HTML and CSS.
        You always summarize texts into bullet points containing the most important details.
        At the end add an interactive button with JavaScript so the user can approve the report,
        and if the user clicks the button, show a pop-up text.
     ''',
    #tools=[], 
    max_iter=10,
    llm=crewai_llm,
    allow_delegation=False, verbose=True)

## Task
task_html = crewai.Task(
    description=prompt,
    agent=agent_html,
    context=[task_py],
    expected_output='''HTML code''')

##### Test single Agent

In [ ]:
crew = crewai.Crew(agents=[agent_html], tasks=[task_html], verbose=False)
res = crew.kickoff(inputs={"user_input": "Number of people survived: 342"})
print("\n---Res---\n", res)

### 4 - Multiple Agents

In [ ]:
crew = crewai.Crew(agents=[agent_sql, agent_py, agent_html], 
                   tasks=[task_sql, task_py, task_html], 
                   process=crewai.Process.sequential,
                   verbose=True)

res = crew.kickoff(inputs={"user_input":"how many people died?"})

print("\n---Res---\n", res)

### 5 - Machine Learning

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=llm, df=dtf, verbose=True, allow_dangerous_code=True,
                                      agent_executor_kwargs={'handle_parsing_errors': True})
agent.invoke("how many people died?")

agent.invoke('''
        You are an experienced data scientist that does machine learning using Python and sckit-learn.
        Take the dataframe and split into train set and test set. 
        Then train a simple classification to predict the column `Survived`.
        Then use the score to evaluate the model predictions.''')